In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Research the existence of the the "zero gamma level"


In [ ]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import cme_expirations as cmeexp
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)

import peakutils
from peakutils.plot import plot as pplot
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate


import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg

import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
from py_vollib.black.greeks import analytical as pyva
import ipdb
import traceback
import pandas_datareader.data as pdr
from barchartacs import plotly_utilities as pu
import dash_bootstrap_components as dbc
import dash_table


### important global variables

In [ ]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()


### Define methods to manipulate datetime and pandas Timestamp objects

In [ ]:
def dt_to_yyyymmdd(datetime_value):
    '''
    convert datetime.datetime object to integer yyyymmdd, 
       like datetime.datetime(2020,11,16) to 20201116
    '''
    y = int(datetime_value.year)
    m = int(datetime_value.month)
    d = int(datetime_value.day)
    return y*100*100 + m*100 + d

def yyyymmdd_to_dt(yyyymmdd):
    '''
    convert integer (or str) of yyyymmdd to a datetime.dateime object
      like 20201116 to datetime.datetime(2020,11,16)
    (The new datetime object will be Timezone naive)
    '''
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])
    return datetime.datetime(y,m,d)

def yyyymmdd_diff(yyyymmdd_low,yyyymmdd_high):
    '''
    Subtract to yyyymmdd dates
    '''
    dt_low = yyyymmdd_to_dt(yyyymmdd_low)
    dt_high = yyyymmdd_to_dt(yyyymmdd_high)
    return (dt_high-dt_low).days

def add_days_to_yyyymmdd(yyyymmdd,days_to_add):
    dt = datetime.datetime.strptime(str(int(str(yyyymmdd))),'%Y%m%d')
    new_dt = dt + datetime.timedelta(days_to_add)
    new_yyyymmdd = int(new_dt.strftime('%Y%m%d'))
    return new_yyyymmdd

def sub_days_from_yyyymmdd(yyyymmdd,days):
    '''
    Subtract days from a yyyymmdd date
    '''
    d = yyyymmdd_to_dt(yyyymmdd)
    d2 = d - datetime.timedelta(days)
    return dt_to_yyyymmdd(d2)

def yyyymmdd_dayofweek(yyyymmdd):
    '''
    Get the day of week of a yyyymdd daste
    '''
    return yyyymmdd_to_dt(yyyymmdd).weekday()


def get_local_peaks_and_valleys(df_in,n,col='close'):
    '''
    :param df - Input DataFrame
    :param n - number of data points that surround peak and valley
    :param col - the data column to observe
    '''
    df = df_in.copy()
    # Find local peaks
    df['valley'] = df.iloc[argrelextrema(df[col].values, np.less_equal, order=n)[0]][col]
    df['peak'] = df.iloc[argrelextrema(df[col].values, np.greater_equal, order=n)[0]][col]
    return df


### Get ES data

In [ ]:
sql = f"""
select * from {futtab} where substring(symbol,1,2)='ES' and settle_date>='20100101'
"""
df_es = pga.get_sql(sql)


In [ ]:
df_es.settle_date.max()

### Get begin and end dates for volskew analysis

In [ ]:
def _get_last_dt(exp_dt,days_back):
    '''
    Change a pandas timestamp into a yyyymmdd integer that is days_back from the timestamp
    '''
    exp_back = exp_dt - datetime.timedelta(days_back)
    exp_back_yyyymmdd =  int(exp_back.strftime('%Y%m%d'))
    return exp_back_yyyymmdd
    
syms = [s for s in df_es.symbol.unique() if s != 'ESY99']
df_es_syms = pd.DataFrame({'symbol':syms})
df_es_syms['sym2'] = df_es_syms.symbol.str.slice(0,2) + df_es_syms.symbol.str.slice(-2,) + df_es_syms.symbol.str.slice(2,3)
df_es_syms = df_es_syms.sort_values('sym2')
df_es_syms.index = range(len(df_es_syms))
df_es_syms = df_es_syms[['symbol']].copy()
df_es_syms['expiry'] = df_es_syms.symbol.apply(cmeexp.get_expiry)
df_es_syms['last_dt'] = df_es_syms.expiry.apply(lambda exp_date:_get_last_dt(exp_date,30))
df_es_syms['first_dt'] = df_es_syms.last_dt.shift(1)
df_es_syms.loc[0,'first_dt'] = add_days_to_yyyymmdd(int(df_es_syms.loc[0,'last_dt']),-90)
df_es_syms.first_dt = df_es_syms.first_dt.astype(int)
df_es_syms.first_dt = df_es_syms.first_dt.apply(lambda fd:add_days_to_yyyymmdd(fd,1))
df_es_syms = df_es_syms[['symbol','first_dt','last_dt']].copy()
df_es_syms

### Graph changes in ATM vol and price to see how often large changes occur around expirations

In [ ]:
df_iv_final_es = pd.read_csv('./temp_folder/df_iv_final_ES.csv')
df_iv_final_es_atm = df_iv_final_es[df_iv_final_es.moneyness==0][['settle_date','atm_iv','close_y']]
df_iv_skew_es = pd.read_csv('./temp_folder/df_iv_skew_ES.csv')


In [ ]:
df_iv_skew_es2 = df_iv_skew_es.merge(df_iv_final_es_atm,how='inner',on='settle_date')
df_iv_skew_es2

In [ ]:
def __skew_delta(r,c):
    F = r.close_y
    K = (1+r[c]) * F
    t = r[colmap['dte']]/365
    iv = r[colmap['iv']] if 'iv' in _COLMAP else np.nan
    cl = r[colmap[opt_col]]
    
df_iv_skew_es2.iloc[:1].apply(
    lambda r: (
        "c",r.close_y,r[0]+r.close_y,
    ),
    axis=1
)

In [ ]:
df_iv_final_es_atm = get_local_peaks_and_valleys(df_iv_final_es_atm,n=100,col='atm_iv')
df_iv_final_es_atm.peak = df_iv_final_es_atm.peak.fillna(0)
df_iv_final_es_atm.valley = df_iv_final_es_atm.valley.fillna(0)

In [ ]:
fig = pu.plotly_plot(
    df_iv_final_es_atm,x_column='settle_date',
    yaxis2_cols=['close_y'],
    plot_title="Show the Dates of Peak Vol Points",
    y_left_label="ATM Vol",
    y_right_label="E-Mini SP Price",
    number_of_ticks_display=30
)
fig

### Compare the dollar amount of open interest in the CME E-mini options to that of SPY

In [ ]:
import yfinance as yf

In [ ]:
tick_spy = yf.Ticker('SPY')

In [ ]:
spy_exps = tick_spy.options

In [ ]:
# Get options for each expiration
df_spy_options = pd.DataFrame()
for exp in spy_exps:
    opt = tick_spy.option_chain(exp)
    opt = pd.DataFrame().append(opt.calls).append(opt.puts)
    opt['expirationDate'] = exp
    df_spy_options = df_spy_options.append(opt, ignore_index=True)


In [ ]:
spy_sum = df_spy_options.openInterest.sum()
spy_dollar_sum = spy_sum * 100
print(f'Dollar volume of open interest in SPY options = {spy_dollar_sum/1000000} (millions)')

In [ ]:
pga = db_info.get_db_info('local')

In [ ]:
es_sql = f"""
select * from {opttab} where settle_date=20210721 and substring(symbol,1,2)='ES'
"""
df_es_db = pga.get_sql(es_sql)
es_sum = df_es_db.open_interest.sum()
es_dollar_sum = es_sum * 5000
print(f'Dollar volume of open interest in SPY options = {es_dollar_sum/1000000} (millions)')

In [ ]:
es_sql_fut = f"""
select * from {futtab} where settle_date=20210721 and substring(symbol,1,2)='ES'
"""
df_es_db_fut = pga.get_sql(es_sql_fut)[['symbol','close']]


In [ ]:
# def delta(r,inc=0.0025,fut_in=None,fut_col='close_y'):
#     fut = r[fut_col] if fut_in is None else fut_in
#     fut_up = fut + inc
#     fut_down = fut - inc
#     price = black.black(r.pc.lower(),fut,r.strike,.02,r.dte/365,r.iv)
#     price_up = black.black(r.pc.lower(),fut_up,r.strike,.02,r.dte/365,r.iv)
#     price_down = black.black(r.pc.lower(),fut_down,r.strike,.02,r.dte/365,r.iv)
#     delta = ((price_up-price)/inc + (price - price_down)/inc)/2
#     return delta

# def gamma(r,inc=0.0025,fut_in=None,fut_col='close_y'):
#     delta_flat = delta(r,inc=inc,fut_in=r[fut_col])
#     delta_up = delta(r,inc=inc,fut_in=r[fut_col]+inc)
#     delta_down = delta(r,inc=inc,fut_in=r[fut_col]-inc)
#     gamma = ((delta_up - delta_flat)/inc + (delta_flat - delta_down)/inc)/2  
#     return gamma

# def vega(r,inc=0.0025,fut_in=None,fut_col='close_y'):
#     return black.greeks.analytical.vega(r.pc.lower(),r[fut_col],r.strike,.02,r.dte/365,r.iv)

def delta(r,inc=0.0025,fut_in=None,fut_col='close_y',rt=.02):
    fut = r[fut_col] if fut_in is None else fut_in
    return pyva.delta(r.pc.lower(),r[fut_col],r.strike,rt,r.dte/365,r.iv)

def gamma(r,inc=0.0025,fut_in=None,fut_col='close_y',rt=.02):
    fut = r[fut_col] if fut_in is None else fut_in
    return pyva.gamma(r.pc.lower(),r[fut_col],r.strike,rt,r.dte/365,r.iv)

def vega(r,inc=0.0025,fut_in=None,fut_col='close_y',rt=.02):
    fut = r[fut_col] if fut_in is None else fut_in
    return pyva.vega(r.pc.lower(),r[fut_col],r.strike,rt,r.dte/365,r.iv)


def lam_pyvol(r,opt_col='close_x',fut_col='close_y'):
    try:
        return implied_volatility.implied_volatility(r[opt_col],r[fut_col],r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1

def get_dte(sym,dt_now):
    exp_date = cmeexp.get_expiry(sym)
    dte = None if exp_date is None else (exp_date - dt_now).days
    return dte



In [ ]:
syms = df_es_db.symbol.unique()
dtes = [get_dte(s,datetime.datetime.now()) for s in syms]
df_dte = pd.DataFrame({'symbol':syms,'dte':dtes})
df_es_db2 = df_es_db.copy()
df_es_db2 = df_es_db2.merge(df_dte,on='symbol',how='inner')

df_es_db2 = df_es_db2.merge(df_es_db_fut,on='symbol',how='inner')
df_es_db2['iv'] = df_es_db2.apply(lam_pyvol,axis=1)
df_es_db2['delta'] = df_es_db2.apply(delta,axis=1)
df_es_db2['gamma'] = df_es_db2.apply(gamma,axis=1)
df_es_db2['gamma_oi'] = df_es_db2.gamma * df_es_db2.open_interest
df_es_db2_gb = df_es_db2[['strike','gamma_oi']].groupby('strike',as_index=False).sum()
fut_value = df_es_db2.close_y.iloc[0]
sdate =  df_es_db2.settle_date.iloc[0]
sdate = f'{str(sdate)[0:4]}-{str(sdate)[4:6]}-{str(sdate)[6:8]}'
fig_oi = pu.plotly_plot(
    df_es_db2_gb[df_es_db2_gb.gamma_oi>=1],
    x_column='strike',bar_plot=True,
    plot_title = f'Gamma X Open Interest vs Strike <br> With the Market at {fut_value} on {sdate}',
    y_left_label='Gamma X Open Interest'
)

In [ ]:
def make_dashtable(
    dtable_id,
    df_in,
    left_align_cols = None,
    editable=False,
    filtering=False,
    columns_to_display=None,
    editable_columns=None,
    max_width='90vw',
    displayed_rows=20,
):
    left_cols = [] if left_align_cols is None else left_align_cols
    ed_cols = [] if editable_columns is None else editable_columns
    # create filter_action
    filter_action = 'none'
    if filtering:
        filter_action = 'fe'
    
    dt = dash_table.DataTable(
        page_current= 0,
        page_size=displayed_rows,
        filter_action=filter_action,
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in left_cols
        ],

        style_as_list_view=False,
        style_table={
            'overflowX':'scroll','overflowY':'scroll',
            'maxWidth': max_width
        } ,
        
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },        
        editable=editable,
        css=[{"selector": "table"},{'selector': '.row', 'rule': 'margin: 0'}],   
        id=dtable_id,
    )
    dt.data=df_in.to_dict('rows')
    dt.columns=[{"name": i, "id": i,'editable': True if i in ed_cols else False} for i in df_in.columns.values]                    
    
    return dt



In [ ]:
def make_title():
    title_div = html.Div(
        [
            html.Center(html.H4("Do Peaks in ATM Vol (and Valleys in Price)")),
            html.Center(html.H4("Occur Near Options Expirations?")),
        ]
    )
    return title_div


In [ ]:
# title_div = make_title(input_list=None)
top_container = dbc.Container(
#     [
#             dbc.Card(
#                 body=True
#             )
#     ]
)

In [ ]:
df_peaks = get_local_peaks_and_valleys(df_iv_final_es_atm,n=100,col='atm_iv')[['settle_date','peak']]
df_peaks = df_peaks[~df_peaks.peak.isna()]
df_peaks.settle_date = df_peaks.settle_date.apply(lambda i:f'{str(i)[0:4]}-{str(i)[4:6]}-{str(i)[6:8]}')
df_peaks.peak = df_peaks.peak.round(4)
df_peaks = df_peaks.rename(columns={'settle_date':'Peak Date','peak':'Peak Value'})
dtable = make_dashtable(
    'peaks_dtab',df_peaks,left_align_cols=['settle_date'])


In [ ]:
def make_app_page(input_list=None):
    layout = dbc.Container(
        [
            dbc.Row(dbc.Col(dcc.Graph(figure=fig))),
            dbc.Row(dbc.Col(dcc.Graph(figure=fig_oi))),
            dbc.Row(
                [
                    dbc.Col(
                        [
                            dbc.Row(
                                dbc.Col(html.H4(html.Center("Dates of Peaks")))
                            ),
                            dbc.Row(dbc.Col(dtable)),

                        ],width={'size':8,'offset':2}
                    )
                ]
            ),
            
        ]
    )
    return layout
        

In [ ]:
RUN_APP=True
if RUN_APP:
    app = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
    # create input
    app.layout = html.Div(
        [
            dbc.Row(dbc.Col(html.Div([make_title()],id='title_div'),width={'size':8,'offset':2})),
            top_container,
            dcc.Loading(children=[
                html.Div([make_app_page()],id='layout_div'),
            ],type='cube',fullscreen=True),
        ]
    )


    app.run_server(debug=False, port=8884)

## END

In [ ]:
def _get_inputs(r,colmap,opt_col='close',fut_col='future'):
    flag = r[colmap['pc']].lower()
    F = r[colmap[fut_col]]
    K = r[colmap['strike']]
    t = r[colmap['dte']]/365
    iv = r[colmap['iv']] if 'iv' in _COLMAP else np.nan
    cl = r[colmap[opt_col]]
    return flag,F,K,t,iv,cl

def __lam_pyvol(r,colmap,rt=.02):
    inputs = _get_inputs(r,colmap)
    try:
        flag = inputs[0]
        F = inputs[1]
        K = inputs[2]
        t = inputs[3]
        iv = inputs[4]
        opt_pr = inputs[5]
        return implied_volatility.implied_volatility(opt_pr,F,K,rt,t,flag)
    except:
        return -1

def __delta(r,colmap,rt=0.02):
    inputs = _get_inputs(r,colmap)
    try:
        flag = inputs[0]
        F = inputs[1]
        K = inputs[2]
        t = inputs[3]
        iv = inputs[4]
        return pyva.delta(flag,F,K,t,rt,iv)
    except:
        print(flag,F,K,t,iv)
        return pyva.delta(flag,F,K,t,rt,iv)

def __gamma(r,colmap,rt=0.02):
    inputs = _get_inputs(r,colmap)
    try:
        flag = inputs[0]
        F = inputs[1]
        K = inputs[2]
        t = inputs[3]
        iv = inputs[4]
        return pyva.gamma(flag,F,K,t,rt,iv)
    except:
        print(flag,F,K,t,iv)
        return pyva.gamma(flag,F,K,t,rt,iv)
    
def __vega(r,colmap,rt=0.02):
    inputs = _get_inputs(r,colmap)
    try:
        flag = inputs[0]
        F = inputs[1]
        K = inputs[2]
        t = inputs[3]
        iv = inputs[4]
        return pyva.vega(flag,F,K,t,rt,iv)
    except:
        print(flag,F,K,t,iv)
        return pyva.vega(flag,F,K,t,rt,iv)    

In [ ]:
for a,b in [(1,3),[2,4]]:
    print(a)
    print(b)

In [ ]:
RECREATE_ALL_DATES = True
if RECREATE_ALL_DATES:
    base_yyyymmdd = 20180101
    es_sql_opt_all_dates = f"""
    select opt.* from {opttab} opt
    where substring(opt.symbol,1,2)='ES' 
    and opt.settle_date > {base_yyyymmdd}
    """
    bg = datetime.datetime.now()
    print(f'beg opt fetch {bg}',end=', ')    
    df_es_opt_db_all_dates = pga.get_sql(es_sql_opt_all_dates)
    print(f'time opt fetch {(datetime.datetime.now() - bg).seconds/60}')

    es_sql_fut_all_dates = f"""
    select fut.* from {futtab} fut
    where substring(fut.symbol,1,2)='ES' 
    and fut.settle_date > {base_yyyymmdd}
    """
    bg = datetime.datetime.now()
    print(f'beg fut fetch {bg}',end=', ')    
    df_es_fut_db_all_dates = pga.get_sql(es_sql_fut_all_dates)[['settle_date','symbol','close']]
    print(f'time fut fetch {(datetime.datetime.now() - bg).seconds/60}')

    df_es_fut_db_all_dates = df_es_fut_db_all_dates.rename(columns={'close':'future'})
    df_es_db_all_dates = df_es_opt_db_all_dates.merge(
        df_es_fut_db_all_dates,
        on=['settle_date','symbol'],
        how='inner'
    )
    
    print(f'df_es_db_all_dates len = {len(df_es_db_all_dates)}')
    df_sym_dates = df_es_db_all_dates[['symbol','settle_date','future']].groupby(
        ['symbol','settle_date'],as_index=False,
    ).count()[['symbol','settle_date']]
    
    print(f'calculate dte values')
    df_sym_dates = df_es_db_all_dates[['symbol','settle_date','future']].groupby(
        ['symbol','settle_date'],as_index=False,
    ).count()[['symbol','settle_date']]
    df_sym_dates['settle_dt'] = [datetime.datetime.strptime(str(d),'%Y%m%d') for d in df_sym_dates.settle_date.values]
    df_sym_dates['dte'] = [get_dte(arr[0],arr[1]) for arr in df_sym_dates[['symbol','settle_dt']].values]

    print(f'Merge dtes with main data in df_es_db_all_dates')
    df_es_db_all_dates = df_es_db_all_dates.merge(
        df_sym_dates[['symbol','settle_date','dte']],
        on=['symbol','settle_date'],
        how='inner'
    )
    
    df_es_db_all_dates = df_es_db_all_dates[df_es_db_all_dates.dte!=0].copy()
        
    bg = datetime.datetime.now()
    print(f'beg iv {bg}',end=', ')
    _COLMAP = {df_es_db_all_dates.columns.values[i]:i for i in range(len(df_es_db_all_dates.columns.values))}
    vals = df_es_db_all_dates.values
    df_es_db_all_dates['iv'] = [__lam_pyvol(r,_COLMAP) for r in vals]
    print(f'time iv {(datetime.datetime.now() - bg).seconds/60}')

    for c,f in [('delta',__delta),('gamma',__gamma),('vega',__vega)]:
        bg = datetime.datetime.now()
        print(f'beg {c} {bg}',end=', ')
        _COLMAP = {df_es_db_all_dates.columns.values[i]:i for i in range(len(df_es_db_all_dates.columns.values))}
        vals = df_es_db_all_dates.values
        df_es_db_all_dates[c] = [f(r,_COLMAP) for r in vals]
        print(f'time {c} {(datetime.datetime.now() - bg).seconds/60}')
    

In [ ]:
def graph_greek(greek_col):
    oi_col = f'{greek_col}_oi'
    if len(greek_col) <= 0:
        df_es_db_all_dates[oi_col] = df_es_db_all_dates.open_interest
    else:
        df_es_db_all_dates[oi_col] = df_es_db_all_dates[greek_col].abs() * df_es_db_all_dates.open_interest

    df_es_db_all_dates_gb_oi = df_es_db_all_dates[['settle_date',oi_col]].groupby(
        'settle_date',as_index=False).sum()
    df_es_db_all_dates_gb_oi = df_es_db_all_dates_gb_oi.merge(df_es_99,on='settle_date',how='inner')
    return pu.plotly_plot(
        df_es_db_all_dates_gb_oi,x_column='settle_date',
        yaxis2_cols=[oi_col],
        plot_title=f"graph {oi_col.replace('_',' ')}"
    )


In [ ]:
for c in ['','gamma','delta','vega']:
    display.display(graph_greek(c))

In [ ]:
df_es_db_all_dates[df_es_db_all_dates.settle_date==20210721]

In [ ]:
df_sym_dates

In [ ]:
df_iv_final_es = pd.read_csv('./temp_folder/df_iv_final_ES.csv')
df_iv_final_es_atm = df_iv_final_es[df_iv_final_es.moneyness==0][['settle_date','atm_iv','close_y']]
df_iv_skew_es = pd.read_csv('./temp_folder/df_iv_skew_ES.csv')


In [ ]:
df_iv_skew_es2

In [ ]:
# pyva.gamma(flag, F, K, t, r, sigma)
gamma_low =  pyva.gamma("c",100,102,30/365,.02,.12)
gamma_high = pyva.gamma("c",100,102,30/365,.02,.30)
gamma_low,gamma_high

In [ ]:
def __skew_greek(r,greek_func,c,rt=0.02):
    F = r.close_y
    K = (1+c) * F
    t = r.dte/365
    iv = r.atm_iv + r[str(c)]
    if np.isnan(iv):
        return np.nan
    opt_price = r[str(c)]
    pc = "p" if c<0 else "c"
    d = greek_func(pc,F,K,t,rt,iv)
    return d

def __skew_delta(r,c,rt=0.02):
    return __skew_greek(r,pyva.delta,c,rt=rt)
def __skew_gamma(r,c,rt=0.02):
    return round(__skew_greek(r,pyva.gamma,c,rt=rt) * 1000,6)
def __skew_vega(r,c,rt=0.02):
    return __skew_greek(r,pyva.vega,c,rt=rt)


In [ ]:
df_iv_skew_es2 = df_iv_skew_es.merge(df_iv_final_es_atm,how='inner',on='settle_date')
df_iv_skew_es2 = df_iv_skew_es2.merge(
    df_sym_dates[['symbol','settle_date','dte']],
    on=['symbol','settle_date'],
    how='inner'
)
df_iv_skew_es2 = df_iv_skew_es2[df_iv_skew_es2.dte>0].copy()

skew_cols = []
for c in df_iv_skew_es2.columns.values:
    try:
        skew_cols.append(float(c))
    except:
        pass

for f in [('gamma',__skew_gamma),('delta',__skew_delta),('vega',__skew_vega)]:    
    for c in skew_cols:
        label = f[0]
        func = f[1]
        df_iv_skew_es2[f'{label}_{c}'] = df_iv_skew_es2.apply(lambda r:func(r,c),axis=1)

In [ ]:
g_cols = [c for c in df_iv_skew_es2.columns.values if 'vega_' in c] 
c1 = df_iv_skew_es2.settle_date>=20210601
c2 = df_iv_skew_es2.settle_date<=20210630
c_all = c1 & c2
df_iv_skew_trans = df_iv_skew_es2[c_all][g_cols].transpose()
df_iv_skew_trans.columns = [str(v) for v in df_iv_skew_es2[c_all].settle_date.values]
df_iv_skew_trans

In [ ]:
dfivst = df_iv_skew_trans[df_iv_skew_trans.columns.values]
# dfivst['settle_date'] = dfivst.index
greek_type = dfivst.index.values[0].split('_')[0]
title = f"{greek_type} per % out of money"
pu.plotly_plot(dfivst,x_column=None,plot_title=title,y_left_label=greek_type)
